<a href="https://colab.research.google.com/github/vbhsharma7/Product-Categorization/blob/main/Product_Categorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Product Classification.xlsx.csv")

In [ ]:
df.head(10)

,text,target
0,Customized Romantic Quote Wooden Frame,"desk accessories,stationery and desk accessories"
1,send birthday cake and flower online,"cakes,photo cakes,designer cakes,regular cakes..."
2,Someone I Love Birthday Cake (1 kg),"cakes,photo cakes"
3,pig,none
4,bags,"bags and clutches,laptop bags,handbags,sling b..."
5,rakhi ludhiana,none
6,perfume for dad,"beauty and personal care,deos and perfumes"
7,god idol brass krishna,"home decor,religious and spiritual idols"
8,good wish personalised,none
9,Metal Finish Elephant with Rudraksha Rakhi and...,none


In [ ]:
df.isnull().sum()

text      0
target    0
dtype: int64

In [ ]:
df.duplicated().sum()

0

In [ ]:
df.shape

(44886, 2)

In [ ]:
df['target'].value_counts()

none                                                                                    12923
cakes,designer cakes,photo cakes,regular cakes                                           4853
flowers                                                                                  1212
kitchen and barware,mugs and cups                                                        1080
cakes,photo cakes,designer cakes,regular cakes                                            981
                                                                                        ...  
teddy bears,soft toys,dolls and playsets,dolls and action figures                           1
beauty and personal care,beauty and personal care,deos and perfumes,makeup items            1
candies and toffies,chocolates,home decor,pendants,pendants and necklaces,showpieces        1
showpieces,home decor,religious and spiritual idols                                         1
dry fruits,home decor,puja accessories,religious and spiritu

In [ ]:
df.drop(df[df.target=='none'].index,inplace=True)

In [ ]:
list1=[]

for i in df.index:
    tar=df['target'][i]
    tar_Index=tar.find(',')
    if tar_Index == -1:
        continue
    target_=tar[0:tar_Index]
    list1.append({"text":df['text'][i],"target":target_})
data=pd.DataFrame(list1)
data.head(10)


,text,target
0,Customized Romantic Quote Wooden Frame,desk accessories
1,send birthday cake and flower online,cakes
2,Someone I Love Birthday Cake (1 kg),cakes
3,bags,bags and clutches
4,perfume for dad,beauty and personal care
5,god idol brass krishna,home decor
6,Floral Design Ceramic Mug with Lid,kitchen and barware
7,designer chocolate cakes,cakes
8,dinosaur rakhi,infants and toddlers
9,Beautiful 14 Pieces Tea Play Set,indoor games


In [ ]:
data['target'].nunique()

158

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
nltk.download('punkt') # tokenizer
nltk.download('stopwords') # 'useless words'
nltk.download('wordnet') # a library for finding synonyms antonyms and so on
nltk.download('averaged_perceptron_tagger') # tagging data
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
import string

def filterPunctuations(tokens):
    '''Removes punctuations from the array of tokens and returns the resultant array'''
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    return words

In [ ]:
def processProductName(product):

    # break the sentence into words
    tokens = word_tokenize(product)

    
    #remove punctuation
    tokens = filterPunctuations(tokens)
 
    
    #convert to lower case
    tokens = [w.lower() for w in tokens]
  

    #remove stopwords
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english'))
    words=list(filter(lambda word:word not in stop_words,tokens))   

    
    #lemmatize the words
    wordnet_lemmatizer=WordNetLemmatizer()
    words=[wordnet_lemmatizer.lemmatize(word) for word in words]
 
    
    #unique words
    words=pd.Series(words).drop_duplicates().tolist()
    return words

In [ ]:
data1=[]

for index, row in data.iterrows(): 
    product=row["text"]
    words=processProductName(product)
    data1.append({"text":product,"token_list":words,"token":' '.join(words),"target":row["target"]})
df1=pd.DataFrame(data1)
df1.head(5)

,text,token_list,token,target
0,Customized Romantic Quote Wooden Frame,"[customized, romantic, quote, wooden, frame]",customized romantic quote wooden frame,desk accessories
1,send birthday cake and flower online,"[send, birthday, cake, flower, online]",send birthday cake flower online,cakes
2,Someone I Love Birthday Cake (1 kg),"[someone, love, birthday, cake, kg]",someone love birthday cake kg,cakes
3,bags,[bag],bag,bags and clutches
4,perfume for dad,"[perfume, dad]",perfume dad,beauty and personal care


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X=df1.iloc[:,0]
y=df1.iloc[:,3]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
vectorizer = CountVectorizer()
X_train_cv=vectorizer.fit_transform(X_train)
transformer = TfidfTransformer()
X_train_tfidf = transformer.fit_transform(X_train_cv)
X_test_cv=vectorizer.transform(X_test)
X_test_tfidf = transformer.transform(X_test_cv)

In [ ]:
models={"Naive Bayes":MultinomialNB(),"SVM":SVC(C=1.0, kernel='linear', gamma='auto'),"KNN":KNeighborsClassifier(),"Decision Tree":DecisionTreeClassifier()}

In [ ]:
from sklearn.model_selection import cross_val_score,KFold
from sklearn.metrics import f1_score

cv = KFold(n_splits=10)
scores=[]
for model in models:
    clf=models[model]
    clf.fit(X_train_tfidf, y_train)
    y_pred=clf.predict(X_test_tfidf)
    f1=cross_val_score(clf, X_train_tfidf, y_train, cv=cv,scoring="f1_micro")
    scores.append({"classifier":model,"cross-validated F1 score":f1.mean()})
scores_df=pd.DataFrame(scores)
scores_df.head()

,classifier,cross-validated F1 score
0,Naive Bayes,0.620228
1,SVM,0.874062
2,KNN,0.794073
3,Decision Tree,0.842795


In [ ]:
clf=models["SVM"]
y_pred=clf.predict(X_test_tfidf)
print(f1_score(y_test.values,y_pred,average="micro"))

0.8815542554365824


In [ ]:
def vectorize(value):
    cv=vectorizer.transform(value)
    tfidf = transformer.transform(cv)
    return tfidf

In [ ]:
def predict_category(text):
    X_test=pd.DataFrame([text],columns=["text"])
    results=[]
    for index, row in X_test.iterrows(): 
        product=row["text"]
        words=' '.join(processProductName(product))
        results.append(words)
    X_test=vectorize(results)
    return clf.predict(X_test)[0]

In [ ]:
category=predict_category(input("Enter the product name : "))
print("The predicted category is: ",category)


Enter the product name : dinosaur rakhi
The predicted category is:  infants and toddlers
